In [1]:
import tensorflow as tf
import numpy as np

In [59]:
G1 = tf.Graph()
with G1.as_default():
    a = tf.constant(5, name="input_a")
    b = tf.constant(3, name="input_b")
    c = tf.multiply(a,b,name="mul_c")
    d=tf.subtract(a,b,name="sub_d")
    e = tf.divide(c,d,"div_e")

In [60]:
with tf.Session(graph=G1) as sess:
    result=(sess.run(e))
    #r_c = sess.run(c)
    writer = tf.summary.FileWriter('./my_graph',sess.graph)
    writer.close()

In [61]:
print(result)

7.5


In [5]:
G2 = tf.Graph()
with G2.as_default():
    a = tf.constant([[5,3],[1,2]], name="input_a")
    b = tf.reduce_prod(a,0,name="prod_b")
    c = tf.reduce_sum(a,0,name="sum_c")
    d = tf.add(b,c,name="add_d")

In [6]:
with tf.Session(graph=G2) as sess:
    result=sess.run(d)
print(result)

[11 11]


In [7]:
#test numoy
t_0 = np.array(50, dtype=np.int32)
t_1 = np.array([2,3,4], dtype=np.int64)
t_2 = np.array([2,3,4], dtype=np.int64)

In [8]:
type(t_1[0])

numpy.int64

In [9]:
G3 = tf.Graph()
with G3.as_default():
    v1  = tf.constant(t_1)
    v2 =tf.constant(t_2)
with tf.Session(graph=G3) as sess:
    #print(sess.run(v1==v2)) not workable
    print(sess.run(tf.equal(v1,v2)))
    print (sess.run (v1>v2))
    #run the two operation as a list
    print ( sess.run ([ tf.equal(v1,v2), v1>v2        ] ) )

[ True  True  True]
[False False False]
[array([ True,  True,  True], dtype=bool), array([False, False, False], dtype=bool)]


In [10]:
#Feed dictionary
#useful when inserting dummy value during debugging

G4 = tf.Graph()
with G4.as_default():
    a = tf.add(2,5)
    b = tf.multiply(a,3)
    
#setup a dictionary to replace the calculation in "a"
replace_dict = {a:15}

with tf.Session(graph=G4) as sess:
    print ("with original run:")
    print(sess.run(b))
    print ("with replacement of a")
    print(sess.run(b,feed_dict=replace_dict))
    

with original run:
21
with replacement of a
45


In [11]:
#demo of interactive session, use wisely....
sess =tf.Session(graph=G4) 
with sess.as_default():
    print(a.eval())
sess.close()


7


In [12]:
#placeholder demo
G5 = tf.Graph()
with G5.as_default():
    a=tf.placeholder(tf.int32,shape=[None],name="my_input")
    b=tf.reduce_prod(a,name="prod_b")
    c = tf.reduce_sum(a,name="sum_c")
    #finish off the graph
    d=tf.add(b,c,name="add_d")


In [13]:
input_dict = {a:np.random.randint(1,6,size=(6))}
print (input_dict)

{<tf.Tensor 'my_input:0' shape=(?,) dtype=int32>: array([4, 4, 2, 4, 4, 4])}


In [14]:
with tf.Session(graph=G5) as sess:
    print(sess.run(d,feed_dict=input_dict))
    print(sess.run([tf.shape(a), tf.shape(b),tf.shape(c),tf.shape(d)],feed_dict=input_dict))
    print(sess.run([b,c],feed_dict=input_dict))

2070
[array([6], dtype=int32), array([], dtype=int32), array([], dtype=int32), array([], dtype=int32)]
[2048, 22]


In [15]:
input_dict = {a:np.random.randint(1,6,size=(6))}
print (input_dict)

{<tf.Tensor 'my_input:0' shape=(?,) dtype=int32>: array([2, 5, 1, 2, 4, 1])}


In [16]:
input_dict[a]

array([2, 5, 1, 2, 4, 1])

In [39]:
#Variable
G6 = tf.Graph()
#No values below 2 std dev or above 2 std dev produced away from mean
with G6.as_default():
    my_var=tf.Variable(3,name="myvar")
    my_var_2 = my_var.assign(my_var*2)
    trunRandom =  tf.truncated_normal([2,2],mean=2.0,stddev=2)
    random_var = tf.Variable(trunRandom)
    init = tf.global_variables_initializer()

In [49]:
with tf.Session(graph=G6) as sess:
    sess.run(init)
    print(sess.run(my_var_2))
    print(sess.run(my_var_2))
    print(sess.run(my_var.assign_add(1)))
    print(sess.run(my_var_2))
    #reset variable
    sess.run(init)
    print(sess.run(my_var_2))

6
12
13
26
6


In [54]:
#name scope
G7=tf.Graph()
with G7.as_default():
    with tf.name_scope("Scope_A"):
        a=tf.add(1,2,name="A_add")
        b=tf.multiply(a,3,name="A_mul")
    with tf.name_scope("Scope_B"):
        c=tf.add(4,5,name="B_add")
        d=tf.multiply(c,6,name="B_mul")
    e=tf.add(b,d,name="output")
    

In [55]:
writer = tf.summary.FileWriter('./name_scope_1',graph=G7)
writer.close()

In [63]:
#named scope more complicated
G8=tf.Graph()
with G8.as_default():
    in_1=tf.placeholder(tf.float32,shape=[],name='input_a')
    in_2=tf.placeholder(tf.float32,shape=[],name='input_b')
    const = tf.constant(3,dtype=tf.float32,name="static_value")
    with tf.name_scope("Transformation"):
        with tf.name_scope("A"):
            A_mul=tf.multiply(in_1,const)
            A_out = tf.subtract(A_mul,in_1)
            
        with tf.name_scope("B"):
            B_mul=tf.multiply(in_2,const)
            B_out = tf.subtract(B_mul,in_2)
        with tf.name_scope("C"):
            C_div = tf.divide(A_out,B_out)
            C_out = tf.add(C_div,const)
        with tf.name_scope("D"):
            D_div = tf.divide(B_out,A_out)
            D_out = tf.add(D_div,const)
            out=tf.maximum(C_out,D_out)

In [64]:
writer = tf.summary.FileWriter("./name_scope_2",graph=G8)
writer.close()